In [2]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor
from sklearn.model_selection import KFold, StratifiedKFold
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.metrics import fbeta_score
from sklearn.ensemble import IsolationForest

# Angenommen, Ihre Daten sind in einem DataFrame df
df = pd.read_csv('../data/GeneralDatensatz18-21ohneGeo-mitLockdown_mitCorona.csv', sep=';')

X = df[['UMONAT','USTUNDE','UWOCHENTAG','UART','USTRZUSTAND','BEZ','UTYP1','ULICHTVERH','IstRad','IstPKW','IstFuss','IstKrad','IstGkfz','IstSonstige', 'LOCKDOWN', 'COVID']]
y = df['UKATEGORIE'].isin([1, 2]).astype(int)

# Initialisieren Sie KFold
sf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

# Initialisieren Sie Listen, um die Ergebnisse zu speichern


for i in range(300, 5000, 50):
    fbeta_scores_SMOTE = []
    # Loop über jeden Split
    for train_index, test_index in sf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
        # Anwendung von SMOTE
        sm = SMOTE(random_state=42)
        X_res, y_res = sm.fit_resample(X_train, y_train)
    
        # Trainieren Sie den Klassifikator
        knn = KNeighborsClassifier(n_neighbors=i, weights='distance')
        knn.fit(X_res, y_res)
    
        # Vorhersagen auf den Testdaten machen
        y_pred = knn.predict(X_test)
        y_pred_train = knn.predict(X_train)
    
        # F-beta-Score berechnen und zur Liste hinzufügen
        fbeta = fbeta_score(y_test, y_pred, beta=2)
        fbeta_scores_SMOTE.append(fbeta)
    
        
    # print(f"SMOTE fbeta scores für {i} neighbors (Train und Test): ", fbeta_train, fbeta_test)

# Durchschnittlichen F-beta-Score über alle Folds ausgeben
    print(f"Average F-beta score for {i} neighbors:" , pd.Series(fbeta_scores_SMOTE).mean())





Average F-beta score for 4500 neighbors: 0.42563504763316795
Average F-beta score for 4501 neighbors: 0.4252882533000898
Average F-beta score for 4502 neighbors: 0.42542492139963395


KeyboardInterrupt: 